In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Notebook magic

In [1]:
from IPython.core.magic import Magics, magics_class, line_cell_magic
from IPython.core.magic import cell_magic, register_cell_magic, register_line_magic
from IPython.core.magic_arguments import argument, magic_arguments, parse_argstring
import subprocess
import os

In [2]:
@magics_class
class PyboardMagic(Magics):
    @cell_magic
    @magic_arguments()
    @argument('-skip')
    @argument('-unix')
    @argument('-pyboard')
    @argument('-file')
    @argument('-data')
    @argument('-time')
    @argument('-memory')
    def micropython(self, line='', cell=None):
        args = parse_argstring(self.micropython, line)
        if args.skip: # doesn't care about the cell's content
            print('skipped execution')
            return None # do not parse the rest
        if args.unix: # tests the code on the unix port. Note that this works on unix only
            with open('/dev/shm/micropython.py', 'w') as fout:
                fout.write(cell)
            proc = subprocess.Popen(["../micropython/ports/unix/micropython-2", "/dev/shm/micropython.py"], 
                                    stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            print(proc.stdout.read().decode("utf-8"))
            print(proc.stderr.read().decode("utf-8"))
            return None
        if args.file: # can be used to copy the cell content onto the pyboard's flash
            spaces = "    "
            try:
                with open(args.file, 'w') as fout:
                    fout.write(cell.replace('\t', spaces))
                    printf('written cell to {}'.format(args.file))
            except:
                print('Failed to write to disc!')
            return None # do not parse the rest
        if args.data: # can be used to load data from the pyboard directly into kernel space
            message = pyb.exec(cell)
            if len(message) == 0:
                print('pyboard >>>')
            else:
                print(message.decode('utf-8'))
                # register new variable in user namespace
                self.shell.user_ns[args.data] = string_to_matrix(message.decode("utf-8"))
        
        if args.time: # measures the time of executions
            pyb.exec('import utime')
            message = pyb.exec('t = utime.ticks_us()\n' + cell + '\ndelta = utime.ticks_diff(utime.ticks_us(), t)' + 
                               "\nprint('execution time: {:d} us'.format(delta))")
            print(message.decode('utf-8'))
        
        if args.memory: # prints out memory information 
            message = pyb.exec('from micropython import mem_info\nprint(mem_info())\n')
            print("memory before execution:\n========================\n", message.decode('utf-8'))
            message = pyb.exec(cell)
            print(">>> ", message.decode('utf-8'))
            message = pyb.exec('print(mem_info())')
            print("memory after execution:\n========================\n", message.decode('utf-8'))

        if args.pyboard:
            message = pyb.exec(cell)
            print(message.decode('utf-8'))

ip = get_ipython()
ip.register_magics(PyboardMagic)

## pyboard

In [57]:
import pyboard
pyb = pyboard.Pyboard('/dev/ttyACM0')
pyb.enter_raw_repl()

In [9]:
pyb.exit_raw_repl()
pyb.close()

In [58]:
%%micropython -pyboard 1

import utime
import ulab as np

def timeit(n=1000):
    def wrapper(f, *args, **kwargs):
        func_name = str(f).split(' ')[1]
        def new_func(*args, **kwargs):
            run_times = np.zeros(n, dtype=np.uint16)
            for i in range(n):
                t = utime.ticks_us()
                result = f(*args, **kwargs)
                run_times[i] = utime.ticks_diff(utime.ticks_us(), t)
            print('{}() execution times based on {} cycles'.format(func_name, n, (delta2-delta1)/n))
            print('\tbest: %d us'%np.min(run_times))
            print('\tworst: %d us'%np.max(run_times))
            print('\taverage: %d us'%np.mean(run_times))
            print('\tdeviation: +/-%.3f us'%np.std(run_times))            
            return result
        return new_func
    return wrapper

def timeit(f, *args, **kwargs):
    func_name = str(f).split(' ')[1]
    def new_func(*args, **kwargs):
        t = utime.ticks_us()
        result = f(*args, **kwargs)
        print('execution time: ', utime.ticks_diff(utime.ticks_us(), t), ' us')
        return result
    return new_func

__END_OF_DEFS__

# scipy.signal

This module defines the single function:

1. [scipy.signal.sosfilt](#sosfilt)

## sosfilt

`scipy`: https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.sosfilt.html 

Filter data along one dimension using cascaded second-order sections.

The function takes two positional arguments, `sos`, the filter segments of length 6, and the one-dimensional, uniformly sampled data set to be filtered. Returns the filtered data, or the filtered data and the final filter delays, if the `zi` keyword arguments is supplied. The keyword argument must be a float `ndarray` of shape `(n_sections, 2)`. If `zi` is not passed to the function, the initial values are assumed to be 0.

In [7]:
%%micropython -unix 1

from ulab import numpy as np
from ulab import scipy as spy

x = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
sos = [[1, 2, 3, 1, 5, 6], [1, 2, 3, 1, 5, 6]]
y = spy.signal.sosfilt(sos, x)
print('y: ', y)

y:  array([0.0, 1.0, -4.0, 24.0, -104.0, 440.0, -1728.0, 6532.000000000001, -23848.0, 84864.0], dtype=float)




In [11]:
%%micropython -unix 1

from ulab import numpy as np
from ulab import scipy as spy

x = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
sos = [[1, 2, 3, 1, 5, 6], [1, 2, 3, 1, 5, 6]]
# initial conditions of the filter
zi = np.array([[1, 2], [3, 4]])

y, zf = spy.signal.sosfilt(sos, x, zi=zi)
print('y: ', y)
print('\n' + '='*40 + '\nzf: ', zf)

y:  array([4.0, -16.0, 63.00000000000001, -227.0, 802.9999999999999, -2751.0, 9271.000000000001, -30775.0, 101067.0, -328991.0000000001], dtype=float)

zf:  array([[37242.0, 74835.0],
	 [1026187.0, 1936542.0]], dtype=float)


